In [1]:
import torch.nn   as nn
import GM_library as GML
import numpy      as np
import torch

from os                   import path, listdir
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps, which can be different
n_denoising_steps = 10

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Define box shape
L = [20, 20, 20]

# Target to generate new crystals
target = 'GM_EPA'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_EMP'

input_folder  = 'models'
target_folder = f'{input_folder}/{target}'
model_name    = f'{target_folder}/model.pt'

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [3]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset    = torch.load(dataset_name_std)
parameters = torch.load(parameters_name_std)

# Assigning parameters accordingly
target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters

# Defining target factor
target_factor = target_std / scale

# Loading the model

In [5]:
# Determine number of features in dataset
n_features = dataset[0].num_node_features

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
edge_model = GML.eGCNN(n_features, dropout_edge).to(device)
print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(4, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 4)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=4, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


# Generating new cystals

In [ ]:
edge_model.eval()
node_model.eval()

# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
    # Initialize the gradient of the optimizers
    node_optimizer.zero_grad()
    edge_optimizer.zero_grad()

    # Diffuse the graph with some noise
    diffused_graph = GML.get_random_graph(n_nodes, n_features)  # CHECK THIS FUNCTION

    # Denoise the diffused graph
    denoised_graph = diffused_graph.clone()
    for t in range(n_denoising_steps):
        # Perform a single forward pass for predicting node features
        out_x = node_model(diffused_graph.x, 
                           diffused_graph.edge_index,
                           diffused_graph.edge_attr)

        # Define x_i and x_j as features of every corresponding pair of nodes (same order than attributes)
        x_i = diffused_graph.x[diffused_graph.edge_index[0]]
        x_j = diffused_graph.x[diffused_graph.edge_index[1]]

        # Perform a single forward pass for predicting edge attributes
        out_attr = edge_model(x_i, x_j)

        # Construct noise graph
        noise_graph = Data(x=out_x, edge_index=diffused_graph.edge_index, edge_attr=out_attr.ravel())

        # Denoise the graph with the predicted noise
        denoised_graph = GML.denoising_step(denoised_graph, noise_graph, t, n_denoising_steps)
    
    # Append generated graph
    predicted_dataset.append(denoised_graph)